In [1]:
import numpy as np
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [47]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
from pathlib import Path

In [4]:
with Path("input.txt").open("r", encoding="utf-8") as f:
    text = f.read()

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab = ''.join(chars)
# print(vocab_size)
# print(vocab)

In [6]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

In [25]:
# take a string and output a list of ints
encode = lambda s: [stoi[c] for c in s]
decode = lambda e: "".join([itos[i] for i in e])
# print(encode("hii there"))
# print(decode(encode("hii there")))

In [26]:
# encode(text)

In [27]:
data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:1000])

In [28]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [29]:
torch.manual_seed(1337)
batch_size = 32
block_size = 8
max_iters = 5000
eval_interval = 300
learning_rate = 1e-3
eval_iters = 200
n_embd = 32
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [30]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [31]:
@torch.no_grad() # we don't intend to do backprop so don't store all intermediate variables
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
# torch.randint(len(train_data) - block_size, (batch_size, ))
xb, yb = get_batch("train")
# print(xb)
# print(xb.shape)
# print(yb)
# print(yb.shape)

In [33]:
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
            
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, 16) x (B, 16, T) --> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1)
        
        # Here is what I will communicate to you (between different heads. Gives head unique values).
        v = self.value(x) # (B, T, 16)
        out = wei @ v # (B, T, T) x (B, T, C) = (B, T, C), for dimension B is preserved

        return out

In [40]:

class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # Init at random a learnable embedding matrix
        self.token_embdding_table = nn.Embedding(vocab_size, n_embd)
        # Each position from 0 to block_size - 1 will
        # get its own positional embedding added on as an offset[
        self.position_embdding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        tok_emb = self.token_embdding_table(idx) # (B, T, C)
        pos_emb = self.position_embdding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_emb + pos_emb # pos_emb is broadcasted across each b in B for tok_emb
        x = self.sa_head(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            # print(i)
            # Get the prediction for the next token given the previous token(s)
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            # Select the predictions only
            # print('before')
            # print(logits.shape)
            # print(logits)
            logits = logits[:, -1, :] # (B, C)
            # print('after')
            # print(logits.shape)
            # print(logits)
            # Run the logits (value of embedding vector weights) through
            # softmax to get probability distribution
            probs = F.softmax(logits, dim=-1) # (B, C)
            # Sample from the distribution to get an index of a vector (a character)
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # Concatenate the idx to the current idx to create the next
            # idx vector to pass into `self` (forward pass function)
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            # print()
        return idx

In [41]:
model = BigramLanguageModel()
m = model.to(device)
logits, loss = m(xb, yb)
# print(logits.shape)
# print(loss)
# print(logits[:, -1])

In [42]:
# emb will lookup and return a tensor of shape (m, vocab_size) or ((B, T), vocab_size)
# of the vectors corresponding to the indices of the tensor you pass in
# val = emb(torch.tensor((5, 39, 2, 58, 29)))
# print(val.shape)

In [43]:
in_ = torch.zeros((1, 1), dtype=torch.long)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[0]])

YqjSw?ccZnVwdC;Kn'YTG:Q';LtFHlv,YAhCUv,pIcQjysBcaCc?dGhMpZmc3UkiYh
B'smO;KO?e'WnXaVjZU-Of!UQnvBLrMZw


In [44]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for iter_ in range(max_iters):
    if iter_ % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter_} train_loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    # print(loss.item())
print(loss.item())

step 0 train_loss 4.2075, val loss 4.2031
step 200 train_loss 3.0338, val loss 3.0589
step 400 train_loss 2.7699, val loss 2.7730
step 600 train_loss 2.6412, val loss 2.6442
step 800 train_loss 2.5687, val loss 2.5732
step 1000 train_loss 2.5109, val loss 2.5418
step 1200 train_loss 2.4978, val loss 2.4981
step 1400 train_loss 2.4704, val loss 2.4757
step 1600 train_loss 2.4537, val loss 2.4744
step 1800 train_loss 2.4438, val loss 2.4568
step 2000 train_loss 2.4443, val loss 2.4524
step 2200 train_loss 2.4484, val loss 2.4411
step 2400 train_loss 2.4331, val loss 2.4319
step 2600 train_loss 2.4224, val loss 2.4255
step 2800 train_loss 2.4224, val loss 2.4326
step 3000 train_loss 2.4078, val loss 2.4313
step 3200 train_loss 2.4054, val loss 2.4137
step 3400 train_loss 2.4015, val loss 2.4131
step 3600 train_loss 2.4057, val loss 2.4012
step 3800 train_loss 2.3942, val loss 2.4120
step 4000 train_loss 2.3804, val loss 2.4133
step 4200 train_loss 2.3840, val loss 2.4102
step 4400 train_l

In [46]:
in_ = torch.zeros((1, 1), dtype=torch.long, device=device)
print(in_)
out = m.generate(in_, max_new_tokens=100)
# print(out)
print(decode(out[0].tolist()))

tensor([[0]])

HINULHAnces ake tofimat he ds. oors ve's tis hiscs. Yous in?

Whily quur
Sane,
FHou ark meeanld nsow
